
# TIANSHOU

In [3]:
import datetime
import os
import pprint

import numpy as np
import torch
import glob

from torch.utils.tensorboard import SummaryWriter
from tianshou.data import Collector, CollectStats, ReplayBuffer, VectorReplayBuffer
from tianshou.exploration import GaussianNoise
from tianshou.highlevel.logger import LoggerFactoryDefault
from tianshou.policy import TD3Policy
from tianshou.policy.base import BasePolicy
from tianshou.trainer import OffpolicyTrainer
from tianshou.utils.net.common import Net
from tianshou.utils.net.continuous import Actor, Critic
from tianshou.env import DummyVectorEnv
from tianshou.utils import TensorboardLogger

import gymnasium as gym

from gymnasium.wrappers import RecordVideo


In [4]:
env = gym.make('LunarLanderContinuous-v2', render_mode="human")

train_envs = DummyVectorEnv([lambda: gym.make('LunarLanderContinuous-v2') for _ in range(10)])
test_envs = DummyVectorEnv([lambda: gym.make('LunarLanderContinuous-v2') for _ in range(10)])

state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
max_action = env.action_space.high[0]

print("Observations shape:", state_shape)
print("Actions shape:", action_shape)
print("Action range:", np.min(env.action_space.low), np.max(env.action_space.high))

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

Observations shape: (8,)
Actions shape: (2,)
Action range: -1.0 1.0


In [5]:
#Model
hidden_sizes = ()
device = "cuda" if torch.cuda.is_available() else "cpu"
net_a = Net(state_shape=state_shape, hidden_sizes=hidden_sizes, device=device)
actor = Actor(net_a, action_shape, max_action=max_action, device=device).to(
    device,
)
actor_optim = torch.optim.Adam(actor.parameters(), lr=1e-3)
net_c1 = Net(
    state_shape=state_shape,
    action_shape=action_shape,
    hidden_sizes=hidden_sizes,
    concat=True,
    device=device,
)
net_c2 = Net(
    state_shape=state_shape,
    action_shape=action_shape,
    hidden_sizes=hidden_sizes,
    concat=True,
    device=device,
)
critic1 = Critic(net_c1, device=device).to(device)
critic1_optim = torch.optim.Adam(critic1.parameters(), lr=1e-3)
critic2 = Critic(net_c2, device=device).to(device)
critic2_optim = torch.optim.Adam(critic2.parameters(), lr=1e-3)

policy: TD3Policy = TD3Policy(
    actor=actor,
    actor_optim=actor_optim,
    critic=critic1,
    critic_optim=critic1_optim,
    critic2=critic2,
    critic2_optim=critic2_optim,
    tau=0.005,
    gamma=0.99,
    exploration_noise=GaussianNoise(sigma=0.1),
    policy_noise=0.2,
    update_actor_freq=2,
    noise_clip=0.5,
    estimation_step=1,
    action_space=env.action_space,
    action_scaling=True
)

# Collector
buffer = VectorReplayBuffer(20000, 10)
train_collector = Collector(policy, train_envs, buffer, exploration_noise=True)
test_collector = Collector(policy, test_envs)
train_collector.reset()
train_collector.collect(n_step=1000, random=True)

# logger
writer = SummaryWriter(log_dir='log/td3_lunar_lander')
logger = TensorboardLogger(writer)

def save_best_fn(policy: BasePolicy) -> None:
    torch.save(policy.state_dict(), 'best_policy.pth')

result = OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=100,
    step_per_epoch=10000,
    step_per_collect=1000,
    episode_per_test=1000,
    batch_size=64,
    save_best_fn=save_best_fn,
    logger=logger,
    update_per_step=0.1,
    test_in_train=False,
    ).run()
pprint.pprint(result)


Epoch #1: 10001it [00:09, 1013.82it/s, env_step=10000, gradient_step=1000, len=68, n/ep=13, n/st=1000, rew=-191.16]                           


Epoch #1: test_reward: -238.972669 ± 86.793901, best_reward: -238.972669 ± 86.793901 in #1


Epoch #2: 10001it [00:08, 1140.13it/s, env_step=20000, gradient_step=2000, len=64, n/ep=13, n/st=1000, rew=-548.36]                           


Epoch #2: test_reward: -543.503579 ± 144.869109, best_reward: -238.972669 ± 86.793901 in #1


Epoch #3: 10001it [00:08, 1130.33it/s, env_step=30000, gradient_step=3000, len=93, n/ep=12, n/st=1000, rew=-313.95]                           


...

Epoch #99: 10001it [00:09, 1110.62it/s, env_step=990000, gradient_step=199000, len=66, n/ep=14, n/st=1000, rew=-447.39]                           


Epoch #99: test_reward: -447.906069 ± 96.146511, best_reward: -87.610979 ± 28.699940 in #69


Epoch #100: 10001it [00:08, 1115.18it/s, env_step=1000000, gradient_step=200000, len=74, n/ep=14, n/st=1000, rew=-323.56]                           


Epoch #100: test_reward: -278.433285 ± 76.535378, best_reward: -87.610979 ± 28.699940 in #69


KeyboardInterrupt: 

In [1]:
# Let's watch its performance!
test_envs.seed(seed)
test_collector.reset()
collector_stats = test_collector.collect(n_episode = 10, render = True)
print(collector_stats)

NameError: name 'test_envs' is not defined

In [7]:
# lunvh tensorboard
from bbrl_utils.notebook import setup_tensorboard

setup_tensorboard("./log/")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 28284), started 4:40:06 ago. (Use '!kill 28284' to kill it.)